# Baroque Ceiling Paintings - DuckDB Analysis

This notebook loads the Parquet database created from NFDI4Culture and CbDD data into a **persistent DuckDB database** for collaborative analysis.

## Data Sources
- **Primary**: NFDI4Culture Knowledge Graph (SPARQL endpoint)
- **Enrichment**: CbDD graphData.json (persons, buildings, rooms)
- **Subject Resolution**: ICONCLASS & Getty AAT

## Tech Stack
- **DuckDB**: Persistent analytical database with SQL interface
- **Polars**: High-performance DataFrame operations
- **Altair**: Declarative visualizations

In [5]:
# =============================================================================
# CELL 1: Setup & Configuration
# =============================================================================
import duckdb
import polars as pl
import altair as alt
import re
from pathlib import Path
from typing import Tuple, Optional
from IPython.display import display as ipython_display

# Configuration
PARQUET_DIR = Path(r"c:/Users/thano/Documents/_Studium/KIT/DataStories/DataStories")
DUCKDB_PATH = PARQUET_DIR / "baroque.duckdb"
PREFIX = "baroque_"

# Table names
TABLES = [
    'paintings', 'persons', 'buildings', 'rooms', 'ensembles',
    'subjects', 'painting_persons', 'painting_subjects',
    'building_persons', 'room_persons'
]

# Enable Altair for larger datasets
alt.data_transformers.disable_max_rows()

print(f"📁 Parquet directory: {PARQUET_DIR}")
print(f"🗄️ DuckDB database: {DUCKDB_PATH}")
print(f"📊 Tables to load: {len(TABLES)}")

📁 Parquet directory: c:\Users\thano\Documents\_Studium\KIT\DataStories\DataStories
🗄️ DuckDB database: c:\Users\thano\Documents\_Studium\KIT\DataStories\DataStories\baroque.duckdb
📊 Tables to load: 10


In [2]:
# =============================================================================
# CELL 2: German Year Parsing Utility
# =============================================================================
# Handles formats like:
#   - "1732" → (1732, 1732, False)
#   - "1732-1742" → (1732, 1742, False)
#   - "um 1732" / "ca. 1579" → (1732, 1732, True)
#   - "um 1732-1742" → (1732, 1742, True)
#   - "17. Jh" / "17. Jahrhundert" → (1600, 1699, False)
#   - "2. Hälfte 17. Jahrhundert" → (1650, 1699, False)
#   - "1. Hälfte 18. Jh." → (1700, 1749, False)
#   - "Anfang 18. Jh." → (1700, 1730, True)
#   - "Ende 17. Jh." → (1670, 1699, True)
#   - "Mitte 18. Jahrhundert" → (1740, 1760, True)
#   - "vor 1750" → (None, 1750, True)
#   - "nach 1700" → (1700, None, True)
#   - "bis 1716" → (None, 1716, False)

def parse_german_year(year_str: Optional[str]) -> Tuple[Optional[int], Optional[int], bool, bool]:
    """
    Parse German year strings into structured data.
    
    Returns:
        (year_start, year_end, is_approximate, is_parsed)
        - year_start: Earliest possible year (or None)
        - year_end: Latest possible year (or None)
        - is_approximate: True if "um", "ca.", "circa", etc.
        - is_parsed: True if successfully parsed, False if unparseable
    """
    if year_str is None or str(year_str).strip() == '':
        return (None, None, False, False)
    
    text = str(year_str).strip().lower()
    original_text = text
    
    # Track approximation markers
    is_approx = False
    approx_markers = ['um ', 'ca. ', 'ca ', 'circa ', 'etwa ', 'wohl ', 'vermutlich ']
    for marker in approx_markers:
        if marker in text:
            is_approx = True
            text = text.replace(marker, '')
    
    # Remove common noise
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    text = text.replace('jhd', 'jh').replace('jahrhundert', 'jh')
    
    # Pattern 1: Simple year "1732"
    match = re.search(r'^(\d{4})$', text.strip())
    if match:
        year = int(match.group(1))
        return (year, year, is_approx, True)
    
    # Pattern 2: Year range "1732-1742" or "1732–1742" or "1732/1742"
    match = re.search(r'(\d{4})\s*[-–/]\s*(\d{4})', text)
    if match:
        return (int(match.group(1)), int(match.group(2)), is_approx, True)
    
    # Pattern 3: Century "17. jh" or "17.jh."
    match = re.search(r'(\d{1,2})\.?\s*jh', text)
    if match:
        century = int(match.group(1))
        base_year = (century - 1) * 100
        
        # Check for half-century modifiers
        if '2. hälfte' in text or 'zweite hälfte' in text:
            return (base_year + 50, base_year + 99, is_approx, True)
        elif '1. hälfte' in text or 'erste hälfte' in text:
            return (base_year, base_year + 49, is_approx, True)
        elif 'anfang' in text or 'beginn' in text or 'früh' in text:
            return (base_year, base_year + 30, True, True)
        elif 'ende' in text or 'spät' in text:
            return (base_year + 70, base_year + 99, True, True)
        elif 'mitte' in text:
            return (base_year + 40, base_year + 60, True, True)
        else:
            return (base_year, base_year + 99, is_approx, True)
    
    # Pattern 4: "vor 1750" (before)
    match = re.search(r'vor\s+(\d{4})', text)
    if match:
        return (None, int(match.group(1)), True, True)
    
    # Pattern 5: "nach 1700" (after) or "ab 1700"
    match = re.search(r'(?:nach|ab)\s+(\d{4})', text)
    if match:
        return (int(match.group(1)), None, True, True)
    
    # Pattern 6: "bis 1716" (until)
    match = re.search(r'bis\s+(\d{4})', text)
    if match:
        return (None, int(match.group(1)), is_approx, True)
    
    # Pattern 7: Extract first 4-digit year as fallback
    match = re.search(r'(\d{4})', text)
    if match:
        year = int(match.group(1))
        # Check if there's a second year
        all_years = re.findall(r'(\d{4})', text)
        if len(all_years) >= 2:
            years = [int(y) for y in all_years]
            return (min(years), max(years), is_approx, True)
        return (year, year, is_approx, True)
    
    # Unparseable
    return (None, None, False, False)


# Test the parser
test_cases = [
    "1732",
    "1732-1742",
    "um 1732-1742",
    "ca. 1579",
    "17. Jh",
    "2. Hälfte 17. Jahrhundert",
    "Anfang 18. Jh.",
    "vor 1750",
    "nach 1700",
    "bis 1716, Weihe 1736",
    "1697-1749, 1782",
    "wohl 18. Jahrhundert",
    "mittelalterlich, 1730-1734",
    None,
    ""
]

print("🧪 Year Parser Test Results:")
print("=" * 80)
print(f"{'Input':<40} {'Start':>6} {'End':>6} {'Approx':>7} {'Parsed':>7}")
print("-" * 80)
for tc in test_cases:
    start, end, approx, parsed = parse_german_year(tc)
    display = str(tc)[:38] if tc else "(None)"
    start_s = str(start) if start else "-"
    end_s = str(end) if end else "-"
    print(f"{display:<40} {start_s:>6} {end_s:>6} {str(approx):>7} {str(parsed):>7}")

🧪 Year Parser Test Results:
Input                                     Start    End  Approx  Parsed
--------------------------------------------------------------------------------
1732                                       1732   1732   False    True
1732-1742                                  1732   1742   False    True
um 1732-1742                               1732   1742    True    True
ca. 1579                                   1579   1579    True    True
17. Jh                                     1600   1699   False    True
2. Hälfte 17. Jahrhundert                  1650   1699   False    True
Anfang 18. Jh.                             1700   1730    True    True
vor 1750                                      -   1750    True    True
nach 1700                                  1700      -    True    True
bis 1716, Weihe 1736                          -   1716   False    True
1697-1749, 1782                            1697   1749   False    True
wohl 18. Jahrhundert                   

In [6]:
# =============================================================================
# CELL 1b: Reconnect to Existing Database (skip if creating fresh)
# =============================================================================
# Use this cell to reconnect to an existing database without rebuilding

if DUCKDB_PATH.exists():
    con = duckdb.connect(str(DUCKDB_PATH))
    print(f"✅ Reconnected to existing database: {DUCKDB_PATH.name}")
    
    # Show table counts
    tables_df = con.execute("SHOW TABLES").pl()
    print(f"📋 Found {len(tables_df)} tables:")
    for row in tables_df.iter_rows(named=True):
        count = con.execute(f"SELECT COUNT(*) FROM {row['name']}").fetchone()[0]
        print(f"   • {row['name']:<25}: {count:>6,} rows")
else:
    print(f"⚠️ Database not found. Run cells 2-5 to create it.")
    con = None

✅ Reconnected to existing database: baroque.duckdb
📋 Found 10 tables:
   • building_persons         :  1,983 rows
   • buildings                :  1,260 rows
   • ensembles                :     32 rows
   • painting_persons         :    329 rows
   • painting_subjects        :    428 rows
   • paintings                :    100 rows
   • persons                  :  2,831 rows
   • room_persons             :  4,676 rows
   • rooms                    :  2,376 rows
   • subjects                 :    284 rows


In [5]:
# =============================================================================
# CELL 3: Create Persistent DuckDB Database
# =============================================================================
# This creates/opens the baroque.duckdb file for persistent storage

# Close any existing connection first
try:
    con.close()
    print("🔌 Closed existing connection")
except:
    pass

# Remove existing database to recreate fresh (comment out to append)
if DUCKDB_PATH.exists():
    try:
        DUCKDB_PATH.unlink()
        print(f"🗑️ Removed existing database: {DUCKDB_PATH.name}")
    except PermissionError:
        print(f"⚠️ Could not remove existing database (in use). Will overwrite tables.")

# Connect to persistent database
con = duckdb.connect(str(DUCKDB_PATH))
print(f"✅ Connected to DuckDB: {DUCKDB_PATH.name}")

# Register the year parser as Python UDFs with special null handling
# null_handling='special' allows the function to receive and return NULL values
con.create_function(
    'parse_year_start', 
    lambda x: parse_german_year(x)[0], 
    ['VARCHAR'], 
    'INTEGER',
    null_handling='special'
)
con.create_function(
    'parse_year_end', 
    lambda x: parse_german_year(x)[1], 
    ['VARCHAR'], 
    'INTEGER',
    null_handling='special'
)
con.create_function(
    'parse_year_approx', 
    lambda x: parse_german_year(x)[2], 
    ['VARCHAR'], 
    'BOOLEAN',
    null_handling='special'
)
con.create_function(
    'parse_year_success', 
    lambda x: parse_german_year(x)[3], 
    ['VARCHAR'], 
    'BOOLEAN',
    null_handling='special'
)

print("✅ Registered year parsing UDFs (with null handling)")

🔌 Closed existing connection
🗑️ Removed existing database: baroque.duckdb
✅ Connected to DuckDB: baroque.duckdb
✅ Registered year parsing UDFs (with null handling)


In [6]:
# =============================================================================
# CELL 4: Load Parquet Tables into DuckDB with Year Parsing
# =============================================================================

def load_parquet_table(con: duckdb.DuckDBPyConnection, table_name: str) -> int:
    """Load a Parquet file into DuckDB as a table."""
    parquet_path = PARQUET_DIR / f"{PREFIX}{table_name}.parquet"
    
    if not parquet_path.exists():
        print(f"   ⚠️ {table_name}: File not found")
        return 0
    
    # For paintings table, add parsed year columns
    if table_name == 'paintings':
        con.execute(f"""
            CREATE TABLE {table_name} AS
            SELECT 
                *,
                parse_year_start(year) AS year_start,
                parse_year_end(year) AS year_end,
                parse_year_approx(year) AS year_is_approximate,
                parse_year_success(year) AS year_parsed
            FROM read_parquet('{parquet_path}')
        """)
    else:
        con.execute(f"""
            CREATE TABLE {table_name} AS
            SELECT * FROM read_parquet('{parquet_path}')
        """)
    
    row_count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    return row_count


print("📥 Loading Parquet tables into DuckDB...")
print("=" * 60)

table_stats = {}
for table in TABLES:
    row_count = load_parquet_table(con, table)
    table_stats[table] = row_count
    status = "✅" if row_count > 0 else "⚠️"
    print(f"   {status} {table:<25}: {row_count:>8,} rows")

print("=" * 60)
total_rows = sum(table_stats.values())
print(f"📊 Total: {total_rows:,} rows across {len([t for t in table_stats if table_stats[t] > 0])} tables")

📥 Loading Parquet tables into DuckDB...
   ✅ paintings                :      100 rows
   ✅ persons                  :    2,831 rows
   ✅ buildings                :    1,260 rows
   ✅ rooms                    :    2,376 rows
   ✅ ensembles                :       32 rows
   ✅ subjects                 :      284 rows
   ✅ painting_persons         :      329 rows
   ✅ painting_subjects        :      428 rows
   ✅ building_persons         :    1,983 rows
   ✅ room_persons             :    4,676 rows
📊 Total: 14,299 rows across 10 tables


In [7]:
# =============================================================================
# CELL 5: Create Indexes for Performance
# =============================================================================

print("🔧 Creating indexes for query performance...")

indexes = [
    # Paintings table
    ("idx_paintings_nfdi", "paintings", "nfdi_uri"),
    ("idx_paintings_cbdd", "paintings", "cbdd_painting_id"),
    ("idx_paintings_building", "paintings", "building_id"),
    ("idx_paintings_room", "paintings", "room_id"),
    ("idx_paintings_year_start", "paintings", "year_start"),
    ("idx_paintings_state", "paintings", "location_state"),
    
    # Junction tables
    ("idx_pp_nfdi", "painting_persons", "nfdi_uri"),
    ("idx_pp_person", "painting_persons", "person_id"),
    ("idx_ps_nfdi", "painting_subjects", "nfdi_uri"),
    ("idx_ps_subject", "painting_subjects", "subject_uri"),
    ("idx_bp_building", "building_persons", "building_id"),
    ("idx_bp_person", "building_persons", "person_id"),
    ("idx_rp_room", "room_persons", "room_id"),
    ("idx_rp_person", "room_persons", "person_id"),
    
    # Entity tables
    ("idx_persons_id", "persons", "person_id"),
    ("idx_buildings_id", "buildings", "building_id"),
    ("idx_rooms_id", "rooms", "room_id"),
    ("idx_subjects_uri", "subjects", "subject_uri"),
]

for idx_name, table, column in indexes:
    try:
        con.execute(f"CREATE INDEX {idx_name} ON {table}({column})")
        print(f"   ✅ {idx_name}")
    except Exception as e:
        if "already exists" in str(e).lower():
            print(f"   ⏭️ {idx_name} (exists)")
        elif "does not exist" in str(e).lower() or "not found" in str(e).lower():
            print(f"   ⚠️ {idx_name} - table/column not found")
        else:
            print(f"   ❌ {idx_name}: {e}")

print("\n✅ Indexing complete")

🔧 Creating indexes for query performance...
   ✅ idx_paintings_nfdi
   ✅ idx_paintings_cbdd
   ✅ idx_paintings_building
   ✅ idx_paintings_room
   ✅ idx_paintings_year_start
   ✅ idx_paintings_state
   ✅ idx_pp_nfdi
   ✅ idx_pp_person
   ✅ idx_ps_nfdi
   ✅ idx_ps_subject
   ✅ idx_bp_building
   ✅ idx_bp_person
   ✅ idx_rp_room
   ✅ idx_rp_person
   ✅ idx_persons_id
   ✅ idx_buildings_id
   ✅ idx_rooms_id
   ✅ idx_subjects_uri

✅ Indexing complete


In [8]:
# =============================================================================
# CELL 6: Year Parsing Validation
# =============================================================================

print("📅 Year Parsing Statistics:")
print("=" * 60)

year_stats = con.execute("""
    SELECT
        COUNT(*) as total,
        SUM(CASE WHEN year IS NOT NULL AND year != '' THEN 1 ELSE 0 END) as has_year_string,
        SUM(CASE WHEN year_parsed THEN 1 ELSE 0 END) as parsed_successfully,
        SUM(CASE WHEN year IS NOT NULL AND year != '' AND NOT year_parsed THEN 1 ELSE 0 END) as parse_failed,
        SUM(CASE WHEN year_is_approximate THEN 1 ELSE 0 END) as approximate_dates,
        MIN(year_start) as earliest_year,
        MAX(year_end) as latest_year
    FROM paintings
""").fetchone()

total, has_year, parsed, failed, approx, earliest, latest = year_stats

print(f"   Total paintings:        {total:>6}")
print(f"   Has year string:        {has_year:>6} ({100*has_year/total:.1f}%)")
print(f"   ✅ Parsed successfully: {parsed:>6} ({100*parsed/total:.1f}%)")
print(f"   ❌ Parse failed:        {failed:>6} ({100*failed/total:.1f}%)")
print(f"   ~ Approximate dates:    {approx:>6}")
print(f"   📆 Date range:          {earliest} - {latest}")

# Show unparseable years for review
if failed > 0:
    print("\n" + "=" * 60)
    print("⚠️ UNPARSEABLE YEAR STRINGS (for manual review):")
    print("=" * 60)
    
    unparsed = con.execute("""
        SELECT year, COUNT(*) as count
        FROM paintings
        WHERE year IS NOT NULL AND year != '' AND NOT year_parsed
        GROUP BY year
        ORDER BY count DESC
        LIMIT 20
    """).pl()
    
    for row in unparsed.iter_rows(named=True):
        print(f"   [{row['count']:>2}x] {row['year']}")

📅 Year Parsing Statistics:
   Total paintings:           100
   Has year string:            91 (91.0%)
   ✅ Parsed successfully:     91 (91.0%)
   ❌ Parse failed:             0 (0.0%)
   ~ Approximate dates:        15
   📆 Date range:          1370 - 1950


In [9]:
# =============================================================================
# CELL 7: Data Quality Summary
# =============================================================================

print("📊 DATA QUALITY SUMMARY")
print("=" * 70)

# Row counts per table
print("\n📋 Table Row Counts:")
for table in TABLES:
    try:
        count = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
        print(f"   {table:<25}: {count:>8,}")
    except:
        print(f"   {table:<25}: (not loaded)")

# Paintings table coverage
print("\n📋 Paintings Column Coverage:")
coverage = con.execute("""
    SELECT
        COUNT(*) as total,
        SUM(CASE WHEN label IS NOT NULL THEN 1 ELSE 0 END) as has_label,
        SUM(CASE WHEN year_parsed THEN 1 ELSE 0 END) as has_year,
        SUM(CASE WHEN lat IS NOT NULL THEN 1 ELSE 0 END) as has_coords,
        SUM(CASE WHEN imageUrl IS NOT NULL THEN 1 ELSE 0 END) as has_image,
        SUM(CASE WHEN cbdd_painting_id IS NOT NULL THEN 1 ELSE 0 END) as has_cbdd,
        SUM(CASE WHEN building_name IS NOT NULL THEN 1 ELSE 0 END) as has_building,
        SUM(CASE WHEN painters IS NOT NULL AND painters != '' THEN 1 ELSE 0 END) as has_painter,
        SUM(CASE WHEN location_state IS NOT NULL THEN 1 ELSE 0 END) as has_state
    FROM paintings
""").fetchone()

total = coverage[0]
cols = ['label', 'year (parsed)', 'coordinates', 'imageUrl', 'CbDD match', 
        'building_name', 'painters', 'location_state']
for i, col in enumerate(cols):
    val = coverage[i + 1]
    pct = 100 * val / total if total > 0 else 0
    bar = "█" * int(pct / 5) + "░" * (20 - int(pct / 5))
    print(f"   {col:<20}: {bar} {val:>5}/{total} ({pct:>5.1f}%)")

# Foreign key integrity
print("\n🔗 Foreign Key Integrity:")

fk_checks = [
    ("painting_persons → paintings", 
     "SELECT COUNT(*) FROM painting_persons pp LEFT JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri WHERE p.nfdi_uri IS NULL"),
    ("painting_subjects → paintings",
     "SELECT COUNT(*) FROM painting_subjects ps LEFT JOIN paintings p ON ps.nfdi_uri = p.nfdi_uri WHERE p.nfdi_uri IS NULL"),
    ("paintings → buildings",
     "SELECT COUNT(*) FROM paintings p LEFT JOIN buildings b ON p.building_id = b.building_id WHERE p.building_id IS NOT NULL AND b.building_id IS NULL"),
    ("paintings → rooms",
     "SELECT COUNT(*) FROM paintings p LEFT JOIN rooms r ON p.room_id = r.room_id WHERE p.room_id IS NOT NULL AND r.room_id IS NULL"),
]

for desc, query in fk_checks:
    try:
        orphans = con.execute(query).fetchone()[0]
        status = "✅" if orphans == 0 else "⚠️"
        print(f"   {status} {desc}: {orphans} orphan(s)")
    except Exception as e:
        print(f"   ⚠️ {desc}: check failed")

📊 DATA QUALITY SUMMARY

📋 Table Row Counts:
   paintings                :      100
   persons                  :    2,831
   buildings                :    1,260
   rooms                    :    2,376
   ensembles                :       32
   subjects                 :      284
   painting_persons         :      329
   painting_subjects        :      428
   building_persons         :    1,983
   room_persons             :    4,676

📋 Paintings Column Coverage:
   label               : ████████████████████   100/100 (100.0%)
   year (parsed)       : ██████████████████░░    91/100 ( 91.0%)
   coordinates         : ████████████████████   100/100 (100.0%)
   imageUrl            : ████████████████████   100/100 (100.0%)
   CbDD match          : ██████████████░░░░░░    74/100 ( 74.0%)
   building_name       : ██████████████░░░░░░    74/100 ( 74.0%)
   painters            : █████████░░░░░░░░░░░    46/100 ( 46.0%)
   location_state      : ██████████████░░░░░░    74/100 ( 74.0%)

🔗 Foreign Key I

In [10]:
# =============================================================================
# CELL 8: Save and Verify Database
# =============================================================================

# Checkpoint to ensure data is written
con.execute("CHECKPOINT")

# Verify database size
db_size = DUCKDB_PATH.stat().st_size / (1024 * 1024)

print("💾 DATABASE SAVED")
print("=" * 60)
print(f"   📁 Path: {DUCKDB_PATH}")
print(f"   📊 Size: {db_size:.2f} MB")
print(f"   📋 Tables: {len([t for t in TABLES if table_stats.get(t, 0) > 0])}")
print(f"   📝 Total rows: {sum(table_stats.values()):,}")

# List all tables in the database
print("\n📋 Tables in database:")
tables_in_db = con.execute("SHOW TABLES").pl()
for row in tables_in_db.iter_rows(named=True):
    print(f"   • {row['name']}")

print("\n✅ Database ready for collaborative use!")
print(f"   Share the file: {DUCKDB_PATH.name}")

💾 DATABASE SAVED
   📁 Path: c:\Users\thano\Documents\_Studium\KIT\DataStories\DataStories\baroque.duckdb
   📊 Size: 7.51 MB
   📋 Tables: 10
   📝 Total rows: 14,299

📋 Tables in database:
   • building_persons
   • buildings
   • ensembles
   • painting_persons
   • painting_subjects
   • paintings
   • persons
   • room_persons
   • rooms
   • subjects

✅ Database ready for collaborative use!
   Share the file: baroque.duckdb


---
# Analysis Queries

Now that the database is loaded, we can run analytical queries using DuckDB SQL with Polars DataFrames as output.

In [7]:
# =============================================================================
# CELL 9: Geographic Analysis - Paintings by German State
# =============================================================================

print("🗺️ GEOGRAPHIC DISTRIBUTION")
print("=" * 60)

geo_df = con.execute("""
    SELECT 
        location_state,
        COUNT(*) as painting_count,
        COUNT(DISTINCT building_id) as building_count,
        MIN(year_start) as earliest,
        MAX(year_end) as latest,
        ROUND(AVG(year_start), 0) as avg_year
    FROM paintings
    WHERE location_state IS NOT NULL
    GROUP BY location_state
    ORDER BY painting_count DESC
""").pl()

print(geo_df)

# Visualization
if len(geo_df) > 0:
    chart = alt.Chart(geo_df.to_pandas()).mark_bar().encode(
        x=alt.X('painting_count:Q', title='Number of Paintings'),
        y=alt.Y('location_state:N', sort='-x', title='German State'),
        color=alt.Color('painting_count:Q', scale=alt.Scale(scheme='blues'), legend=None),
        tooltip=['location_state', 'painting_count', 'building_count', 'earliest', 'latest']
    ).properties(
        title='Baroque Ceiling Paintings by German State',
        width=500,
        height=400
    )
    ipython_display(chart)

🗺️ GEOGRAPHIC DISTRIBUTION
shape: (12, 6)
┌─────────────────────┬────────────────┬────────────────┬──────────┬────────┬──────────┐
│ location_state      ┆ painting_count ┆ building_count ┆ earliest ┆ latest ┆ avg_year │
│ ---                 ┆ ---            ┆ ---            ┆ ---      ┆ ---    ┆ ---      │
│ str                 ┆ i64            ┆ i64            ┆ i32      ┆ i32    ┆ f64      │
╞═════════════════════╪════════════════╪════════════════╪══════════╪════════╪══════════╡
│ Bayern              ┆ 24             ┆ 14             ┆ 1541     ┆ 1757   ┆ 1639.0   │
│ Baden-Württemberg   ┆ 12             ┆ 10             ┆ 1575     ┆ 1790   ┆ 1721.0   │
│ Thüringen           ┆ 11             ┆ 9              ┆ 1695     ┆ 1792   ┆ 1732.0   │
│ Niedersachsen       ┆ 5              ┆ 5              ┆ 1569     ┆ 1777   ┆ 1650.0   │
│ Sachsen             ┆ 4              ┆ 4              ┆ 1586     ┆ 1699   ┆ 1627.0   │
│ …                   ┆ …              ┆ …              ┆ …        ┆

alt.Chart(...)

In [9]:
# =============================================================================
# CELL 10: Temporal Analysis - Paintings by Decade
# =============================================================================

print("📅 TEMPORAL DISTRIBUTION")
print("=" * 60)

temporal_df = con.execute("""
    SELECT 
        CAST(FLOOR(year_start / 10) * 10 AS INTEGER) as decade,
        COUNT(*) as count,
        CAST(SUM(CASE WHEN year_is_approximate THEN 1 ELSE 0 END) AS INTEGER) as approximate_count
    FROM paintings
    WHERE year_start IS NOT NULL
      AND year_start >= 1500 AND year_start <= 1900
    GROUP BY FLOOR(year_start / 10) * 10
    ORDER BY decade
""").pl()

print(temporal_df)

# Visualization
if len(temporal_df) > 0:
    chart = alt.Chart(temporal_df.to_pandas()).mark_bar().encode(
        x=alt.X('decade:O', title='Decade'),
        y=alt.Y('count:Q', title='Number of Paintings'),
        color=alt.Color('approximate_count:Q', 
                       scale=alt.Scale(scheme='oranges'),
                       title='Approximate Dates'),
        tooltip=['decade', 'count', 'approximate_count']
    ).properties(
        title='Baroque Ceiling Paintings by Decade',
        width=600,
        height=300
    )
    ipython_display(chart)

📅 TEMPORAL DISTRIBUTION
shape: (22, 3)
┌────────┬───────┬───────────────────┐
│ decade ┆ count ┆ approximate_count │
│ ---    ┆ ---   ┆ ---               │
│ i32    ┆ i64   ┆ i32               │
╞════════╪═══════╪═══════════════════╡
│ 1540   ┆ 9     ┆ 2                 │
│ 1560   ┆ 1     ┆ 1                 │
│ 1570   ┆ 3     ┆ 0                 │
│ 1580   ┆ 2     ┆ 0                 │
│ 1600   ┆ 4     ┆ 0                 │
│ …      ┆ …     ┆ …                 │
│ 1750   ┆ 3     ┆ 0                 │
│ 1760   ┆ 4     ┆ 1                 │
│ 1770   ┆ 3     ┆ 1                 │
│ 1780   ┆ 2     ┆ 0                 │
│ 1790   ┆ 1     ┆ 0                 │
└────────┴───────┴───────────────────┘


alt.Chart(...)

In [10]:
# =============================================================================
# CELL 11: Top Painters Analysis
# =============================================================================

print("🎨 TOP PAINTERS")
print("=" * 60)

painters_df = con.execute("""
    SELECT 
        pp.person_name,
        COUNT(DISTINCT pp.nfdi_uri) as painting_count,
        COUNT(DISTINCT p.building_id) as building_count,
        MIN(p.year_start) as active_from,
        MAX(p.year_end) as active_until
    FROM painting_persons pp
    JOIN paintings p ON pp.nfdi_uri = p.nfdi_uri
    WHERE pp.role = 'PAINTER'
    GROUP BY pp.person_name
    HAVING COUNT(DISTINCT pp.nfdi_uri) >= 2
    ORDER BY painting_count DESC
    LIMIT 25
""").pl()

print(painters_df)

# Visualization
if len(painters_df) > 0:
    chart = alt.Chart(painters_df.head(15).to_pandas()).mark_bar().encode(
        x=alt.X('painting_count:Q', title='Number of Paintings'),
        y=alt.Y('person_name:N', sort='-x', title='Painter'),
        color=alt.Color('building_count:Q', 
                       scale=alt.Scale(scheme='viridis'),
                       title='Buildings'),
        tooltip=['person_name', 'painting_count', 'building_count', 'active_from', 'active_until']
    ).properties(
        title='Most Prolific Baroque Ceiling Painters',
        width=500,
        height=400
    )
    ipython_display(chart)

🎨 TOP PAINTERS
shape: (13, 5)
┌────────────────┬────────────────┬────────────────┬─────────────┬──────────────┐
│ person_name    ┆ painting_count ┆ building_count ┆ active_from ┆ active_until │
│ ---            ┆ ---            ┆ ---            ┆ ---         ┆ ---          │
│ str            ┆ i64            ┆ i64            ┆ i32         ┆ i32          │
╞════════════════╪════════════════╪════════════════╪═════════════╪══════════════╡
│ Harms          ┆ 3              ┆ 1              ┆ 1682        ┆ 1714         │
│ Hermann        ┆ 3              ┆ 1              ┆ 1732        ┆ 1742         │
│ Lammers        ┆ 3              ┆ 1              ┆ 1695        ┆ 1697         │
│ Johann Oswald  ┆ 3              ┆ 1              ┆ 1682        ┆ 1714         │
│ Seivert        ┆ 3              ┆ 1              ┆ 1695        ┆ 1697         │
│ …              ┆ …              ┆ …              ┆ …           ┆ …            │
│ Dubuisson      ┆ 2              ┆ 1              ┆ 1747        ┆ 1

alt.Chart(...)

In [11]:
# =============================================================================
# CELL 12: Building Function Analysis
# =============================================================================

print("🏛️ BUILDING FUNCTION ANALYSIS")
print("=" * 60)

building_func_df = con.execute("""
    SELECT 
        building_function,
        COUNT(*) as painting_count,
        COUNT(DISTINCT building_id) as building_count,
        ROUND(AVG(year_start), 0) as avg_year
    FROM paintings
    WHERE building_function IS NOT NULL
    GROUP BY building_function
    ORDER BY painting_count DESC
    LIMIT 20
""").pl()

print(building_func_df)

# Visualization - Pie/Donut chart
if len(building_func_df) > 0:
    chart = alt.Chart(building_func_df.head(10).to_pandas()).mark_arc(innerRadius=50).encode(
        theta=alt.Theta('painting_count:Q'),
        color=alt.Color('building_function:N', title='Building Type'),
        tooltip=['building_function', 'painting_count', 'building_count']
    ).properties(
        title='Paintings by Building Function',
        width=400,
        height=400
    )
    ipython_display(chart)

🏛️ BUILDING FUNCTION ANALYSIS
shape: (20, 4)
┌─────────────────────────────────┬────────────────┬────────────────┬──────────┐
│ building_function               ┆ painting_count ┆ building_count ┆ avg_year │
│ ---                             ┆ ---            ┆ ---            ┆ ---      │
│ str                             ┆ i64            ┆ i64            ┆ f64      │
╞═════════════════════════════════╪════════════════╪════════════════╪══════════╡
│ Hauptresidenz                   ┆ 17             ┆ 12             ┆ 1678.0   │
│ Residenz                        ┆ 16             ┆ 7              ┆ 1629.0   │
│ Lustschloss                     ┆ 8              ┆ 5              ┆ 1718.0   │
│ Schloss                         ┆ 6              ┆ 4              ┆ 1672.0   │
│ Herrenhaus / Rittersitz / Guts… ┆ 4              ┆ 4              ┆ 1735.0   │
│ …                               ┆ …              ┆ …              ┆ …        │
│ Bauernhaus                      ┆ 1              ┆ 1          

alt.Chart(...)

In [12]:
# =============================================================================
# CELL 13: Subject/Theme Analysis (ICONCLASS)
# =============================================================================

print("📚 SUBJECT/THEME ANALYSIS")
print("=" * 60)

subjects_df = con.execute("""
    SELECT 
        s.subject_label,
        s.subject_source,
        COUNT(DISTINCT ps.nfdi_uri) as painting_count
    FROM painting_subjects ps
    JOIN subjects s ON ps.subject_uri = s.subject_uri
    WHERE s.subject_label IS NOT NULL
    GROUP BY s.subject_label, s.subject_source
    ORDER BY painting_count DESC
    LIMIT 30
""").pl()

print(subjects_df)

# Visualization
if len(subjects_df) > 0:
    chart = alt.Chart(subjects_df.head(15).to_pandas()).mark_bar().encode(
        x=alt.X('painting_count:Q', title='Number of Paintings'),
        y=alt.Y('subject_label:N', sort='-x', title='Subject/Theme'),
        color=alt.Color('subject_source:N', title='Source'),
        tooltip=['subject_label', 'subject_source', 'painting_count']
    ).properties(
        title='Most Common Subjects in Baroque Ceiling Paintings',
        width=500,
        height=400
    )
    ipython_display(chart)

📚 SUBJECT/THEME ANALYSIS
shape: (30, 3)
┌─────────────────────────────────┬────────────────┬────────────────┐
│ subject_label                   ┆ subject_source ┆ painting_count │
│ ---                             ┆ ---            ┆ ---            │
│ str                             ┆ str            ┆ i64            │
╞═════════════════════════════════╪════════════════╪════════════════╡
│ ceiling paintings               ┆ GETTY_AAT      ┆ 74             │
│ buildings (structures)          ┆ GETTY_AAT      ┆ 26             │
│ cupids: 'amores', 'amoretti', … ┆ ICONCLASS      ┆ 9              │
│ ornament ~ grotesque            ┆ ICONCLASS      ┆ 7              │
│ birds                           ┆ ICONCLASS      ┆ 6              │
│ …                               ┆ …              ┆ …              │
│ ruler, sovereign                ┆ ICONCLASS      ┆ 2              │
│ garland, wreath                 ┆ ICONCLASS      ┆ 2              │
│ fruits                          ┆ ICONCLASS     

alt.Chart(...)

In [13]:
# =============================================================================
# CELL 14: Painter Collaboration Network
# =============================================================================

print("🤝 PAINTER COLLABORATIONS")
print("=" * 60)
print("(Painters who worked on the same painting)")

collabs_df = con.execute("""
    SELECT 
        pp1.person_name AS painter1,
        pp2.person_name AS painter2,
        COUNT(DISTINCT pp1.nfdi_uri) AS collaborations
    FROM painting_persons pp1
    JOIN painting_persons pp2 
        ON pp1.nfdi_uri = pp2.nfdi_uri 
        AND pp1.person_id < pp2.person_id
    WHERE pp1.role = 'PAINTER' AND pp2.role = 'PAINTER'
    GROUP BY pp1.person_name, pp2.person_name
    HAVING COUNT(DISTINCT pp1.nfdi_uri) >= 1
    ORDER BY collaborations DESC
    LIMIT 20
""").pl()

print(collabs_df)

# Visualization - Collaboration pairs
if len(collabs_df) > 0 and collabs_df['collaborations'].max() > 0:
    df_viz = collabs_df.head(15).to_pandas()
    df_viz['pair'] = df_viz['painter1'].str[:20] + ' + ' + df_viz['painter2'].str[:20]
    
    chart = alt.Chart(df_viz).mark_bar().encode(
        x=alt.X('collaborations:Q', title='Number of Collaborations'),
        y=alt.Y('pair:N', sort='-x', title='Painter Pair'),
        color=alt.Color('collaborations:Q', scale=alt.Scale(scheme='purples'), legend=None),
        tooltip=['painter1', 'painter2', 'collaborations']
    ).properties(
        title='Painter Collaborations',
        width=500,
        height=400
    )
    ipython_display(chart)

🤝 PAINTER COLLABORATIONS
(Painters who worked on the same painting)
shape: (0, 3)
┌──────────┬──────────┬────────────────┐
│ painter1 ┆ painter2 ┆ collaborations │
│ ---      ┆ ---      ┆ ---            │
│ str      ┆ str      ┆ i64            │
╞══════════╪══════════╪════════════════╡
└──────────┴──────────┴────────────────┘


In [14]:
# =============================================================================
# CELL 15: Close Connection (run when done)
# =============================================================================

# Uncomment to close the connection
# con.close()
# print("✅ DuckDB connection closed")

print("💡 Tip: The database file 'baroque.duckdb' can be shared with collaborators.")
print("   They can open it with:")
print("   >>> con = duckdb.connect('baroque.duckdb')")
print("   >>> con.execute('SELECT * FROM paintings LIMIT 5').pl()")

💡 Tip: The database file 'baroque.duckdb' can be shared with collaborators.
   They can open it with:
   >>> con = duckdb.connect('baroque.duckdb')
   >>> con.execute('SELECT * FROM paintings LIMIT 5').pl()
